In [1]:
import pandas as pd
import igraph
import datetime
import random
import csv

In [3]:
networks = '/Volumes/Data2/RST/notebook/Real Time/networks/'

In [4]:
period = 'AM'

In [7]:
od_data = pd.read_csv('/Volumes/Data2/RST/notebook/TTS OD/TTS_TYPE.csv')
od_data['destination'] = od_data['destination'] + 1000
od_data['origin'] = od_data['origin'] + 1000

In [76]:
# G = igraph.Graph.Read_GraphML(f = networks + 'scheduled/2019-11-14_' + period + '.graphml')
#G = igraph.Graph.Read_GraphML(f = '/Volumes/Data2/RST/notebook/networks/' + period + '-TE-16-8.graphml') 

#G = igraph.Graph.Read_GraphML(f = networks + '2019-12-02_' + period + '.graphml')

G = igraph.Graph.Read_GraphML(f = networks + '2019-11-28_' + period + '.graphml')

In [77]:
if period == 'EM':
    depart_hour = '-4-'
    tts_type = 'raw_em'
if period == 'AM':
    depart_hour = '-7-'
    tts_type = 'raw_am'
elif period =='PM':
    depart_hour = '-17-'
    tts_type = 'raw_pm'
elif period =='MD':
    depart_hour = '-11-'
    tts_type = 'raw_md'
elif period =='EV':
    depart_hour = '-20-'
    tts_type = 'raw_ev'

In [78]:
num_int = 1

In [79]:
minute = random.randint(0,9)
minute = minute + num_int * 10
rand_time = depart_hour + str(minute)

In [80]:
node_data = od_data[od_data['type'] == tts_type][['origin', 'destination']].copy()
node_data['origin'] = node_data['origin'].astype(str)
node_data['destination'] = node_data['destination'].astype(str)

node_data['o_node'] = node_data['origin'] + '-0' + rand_time


node_data['d_node'] = node_data['destination'] + '-0-99-99'
node_data = node_data.reset_index()

In [81]:
node_key_table = []

for i in G.vs:
    node_key_table.append([i.index,  i['id']])
node_df_new = pd.DataFrame.from_records(node_key_table, columns = ['index', 'node_id'])

node_df_new['INT_ID'] = node_df_new['node_id'].str.split('-', expand = True)[0].astype(int)

In [82]:
node_df_new

,index,node_id,INT_ID
0,0,13445821-0-7-0,13445821
1,1,13445821-0-7-1,13445821
2,2,13445821-0-7-2,13445821
3,3,13445821-0-7-3,13445821
4,4,13445821-0-7-4,13445821
...,...,...,...
1518196,1518196,13465966-0-11-55,13465966
1518197,1518197,13465966-0-11-56,13465966
1518198,1518198,13465966-0-11-57,13465966
1518199,1518199,13465966-0-11-58,13465966


In [83]:
o_list_new = list(node_df_new[node_df_new['node_id'].isin(node_data['o_node'])]['index'])
d_list_new = list(node_df_new[node_df_new['node_id'].isin(node_data['d_node'])]['index'])

In [85]:
out_all = G.shortest_paths(o_list_new, d_list_new, weights = 'cost',  mode = 'out')

In [86]:
table = []
for i in range(len(o_list_new)):
    temp_lst = out_all[i]
    for j in range(len(d_list_new)):
        table.append([o_list_new[i], d_list_new[j], temp_lst[j]])

In [87]:
temp_df = pd.DataFrame.from_records(table, columns = ['index_o', 'index_d', 'travel_time'])
temp_df = temp_df.merge(node_df_new[['node_id', 'index', 'INT_ID']], left_on = ['index_o'], right_on = ['index'])
temp_df = temp_df.rename(columns = {'node_id':'o_node', 'INT_ID':'origin'})[['o_node', 'index_d', 'origin','travel_time']]
temp_df = temp_df.merge(node_df_new[['node_id', 'index','INT_ID']], left_on = ['index_d'], right_on = ['index'])
temp_df = temp_df.rename(columns = {'node_id':'d_node','INT_ID':'destination'})[['o_node', 'd_node', 'origin','destination','travel_time']]
temp_df = temp_df.merge(node_data[['o_node', 'd_node']])
temp_df = temp_df[temp_df['origin'] != temp_df['destination']]

In [88]:
#temp_df_1 = temp_df

In [89]:
#temp_df16 = temp_df

In [90]:
#temp_df_19_2 = temp_df

In [91]:
temp_df_1128 = temp_df

In [64]:
# temp_df16 = temp_df16.rename(columns = {'travel_time':'travel_time16'})
# temp_df_1 = temp_df_1.rename(columns = {'travel_time':'travel_time19'})
# temp_df_19_2 = temp_df_19_2.rename(columns = {'travel_time':'travel_time19-02'})

In [72]:
# compare = temp_df16[['origin', 'destination', 'travel_time16']].merge(temp_df_1[['origin', 'destination', 'travel_time19']]
#                                                            ).merge(temp_df_19_2[['origin', 'destination', 'travel_time19-02']])

In [92]:
temp_df_1128 = temp_df_1128.rename(columns = {'travel_time':'travel_time19-28'})

compare = compare.merge(temp_df_1128[['origin', 'destination', 'travel_time19-28']])

In [93]:
compare.sum()

origin              9723931.0
destination         8778424.0
travel_time16        401217.0
travel_time19        405706.0
travel_time19-02     532320.0
travel_time19-28     460162.0
dtype: float64

In [94]:
compare

,origin,destination,travel_time16,travel_time19,travel_time19-02,travel_time19-28
0,1250,1001,41.0,38.0,43.0,40.0
1,1540,1001,29.0,27.0,31.0,30.0
2,1211,1003,62.0,71.0,74.0,77.0
3,1534,1003,54.0,48.0,89.0,62.0
4,1105,1004,56.0,55.0,78.0,72.0
...,...,...,...,...,...,...
7433,1397,1007,132.0,134.0,172.0,163.0
7434,1100,1081,44.0,45.0,50.0,45.0
7435,1211,1081,49.0,52.0,54.0,52.0
7436,1423,1081,49.0,51.0,58.0,47.0
